<div style="width: 100%; clear: both;">
    <div style="float: left; width: 20%;">
       <img src="https://joanamengualcom.files.wordpress.com/2021/11/blockstellart-2.png?w=100", align="left">
    </div>
</div>

<div style="float: right; width: 50%;">
    <p style="margin: 0; padding-top: 22px; text-align:right;"> Construcción de mi Criptomoneda con Python</p>
    <p style="margin: 0; text-align:right;">Formación en la Tecnología Blockchain</p>
    <p style="margin: 0; text-align:right; padding-button: 100px;">Joan Amengual</p>
</div>

</div>
<div style="width: 100%; clear: both;">
<div style="width:100%;">&nbsp;</div>

Librerías necesarias para la creación de una Blockchain con Python:

* **datetime**: El módulo datetime proporciona clases para manipular fechas y horas. Si bien la implementación permite operaciones aritméticas con fechas y horas, su principal objetivo es poder extraer campos de forma eficiente para su posterior manipulación o formateo.

* **hashlib**: Este módulo implementa una interfaz común a diferentes algoritmos de hash y resúmenes de mensajes seguros. Están incluidos los algoritmos de hash FIPS seguros SHA1, SHA224, SHA226, SHA384 y SHA512 (definidos en FIPS 180-2) además del algoritmo MD5 de RSA (definido en Internet RFC 1321). Los términos «hash seguro» y «resumen de mensaje» son intercambiables. Los algoritmos más antiguos fueron denominados resúmenes de mensajes. El término moderno es hash seguro.

* **json**: JSON son las siglas de JavaScript Object Notation. JSON es un formato de datos ligero que se utiliza para el intercambio de datos entre varios lenguajes diferentes. Es fácil de leer para los humanos y fácilmente analizado por las máquinas.

* **flask**: Flask es un micro framework web escrito en Python. Se clasifica como un microframework porque no requiere herramientas o librerías particulares. No tiene capa de abstracción de base de datos, validación de formularios, o cualquier otro componente donde las librerías de terceros preexistentes proporcionan funciones comunes. Sin embargo, Flask admite extensiones que pueden añadir funciones a la aplicación como si estuvieran implementadas en el propio Flask. Existen extensiones para mapeadores objeto-relacionales, validación de formularios, manejo de cargas, varias tecnologías de autenticación abierta y varias herramientas relacionadas con el marco común.

* **flask-ngrok**: Una forma sencilla de hacer demostraciones de aplicaciones Flask desde tu máquina. Hace que sus aplicaciones Flask que se ejecutan en localhost estén disponibles en Internet a través de la excelente herramienta ngrok.

* **requests**: Requests es una biblioteca HTTP para el lenguaje de programación Python. El objetivo del proyecto es hacer que las solicitudes HTTP sean más simples y amigables para los humanos. La versión actual es 2.26.0. Las solicitudes se publican bajo la licencia Apache 2.0.

* **uuid**: Este módulo proporciona objetos UUID inmutables (la clase UUID) y las funciones uuid1(), uuid3(), uuid4(), uuid5() para generar UUIDs de las versiones 1, 3, 4 y 5 como se especifica en el RFC 4122.
Si lo único que quieres es un ID único, probablemente debas llamar a uuid1() o a uuid4(). Ten en cuenta que uuid1() puede comprometer la privacidad, ya que crea un UUID que contiene la dirección de red del ordenador. uuid4() crea un UUID aleatorio.

* **urllib.parse**: Este módulo define una interfaz estándar para dividir las cadenas de localizadores uniformes de recursos (URL) en componentes (esquema de direccionamiento, ubicación en la red, ruta, etc.), para volver a combinar los componentes en una cadena de URL, y para convertir una "URL relativa" en una URL absoluta dada una "URL base". El módulo ha sido diseñado para ajustarse a la RFC de Internet sobre Localizadores Uniformes de Recursos Relativos. Admite los siguientes esquemas de URL: *file, ftp, gopher, hdl, http, https, imap, mailto, mms, news, nntp, prospero, rsync, rtsp, rtspu, sftp, shttp, sip, sips, snews, svn, svn+ssh, telnet, wais, ws, wss*.



# Instalaciones

In [15]:
!pip install flask==1.1.2

In [16]:
!pip install requests==2.25.1

In [17]:
!pip install flask-ngrok==0.0.25

# Mi Criptomoneda con Python

In [18]:
# Importación de las librerías
import datetime
import hashlib
import json
import requests
from uuid         import uuid4
from flask        import Flask, jsonify, request
from urllib.parse import urlparse
from flask_ngrok  import run_with_ngrok

Creación de una clase Blockchain que contenga todos los métodos esenciales, como son los siguientes:

* Creación de un nuevo bloque
* Obtención del hash de un bloque
* Protocolo de concenso Proof of Work (PoW)
* Generación del hash de un bloque
* Verificación de la validez de la Blockchain
* Añadir nueva transacción a la cadena de bloques
* Añadir nuevo nodo a la cadena de bloques
* Remplazar la Blockchain por la adecuada

In [19]:
class Blockchain:
    
  def __init__(self):
    """ Constructor de la clase. """

    self.chain = []
    self.transactions = []
    self.create_block(proof = 1, previous_hash = '0')
    self.nodes = set()
      
  def create_block(self, proof, previous_hash):
    """ Creación de un nuevo bloque. 

      Arguments:
        - proof: Nounce del bloque actual.
        - previous_hash: Hash del bloque previo.

      Returns: 
        - block: Nuevo bloque creado. 
      """

    block = { 'index'         : len(self.chain)+1,
              'timestamp'     : str(datetime.datetime.now()),
              'proof'         : proof,
              'previous_hash' : previous_hash,
              'transactions'  : self.transactions}
    self.transactions = []
    self.chain.append(block)
    return block

  def get_previous_block(self):
    """ Obtención del bloque previo de la Blockchain.
    
      Returns:
        - Obtención del último bloque de la Blockchain. """

    return self.chain[-1]
  
  def proof_of_work(self, previous_proof):     
    """ Protocolo de concenso Proof of Work (PoW).
    
      Arguments:
        - previous_proof: Nounce del bloque previo.

      Returns:
        - new_proof: Devolución del nuevo nounce obtenido con PoW. """

    new_proof = 1
    check_proof = False
    while check_proof is False:
        hash_operation = hashlib.sha256(str(new_proof**2 - previous_proof**2).encode()).hexdigest()
        if hash_operation[:4] == '0000':
            check_proof = True
        else: 
            new_proof += 1
    return new_proof
  
  def hash(self, block):
    """ Cálculo del hash de un bloque.
    
    Arguments:
        - block: Identifica a un bloque de la Blockchain.
    
    Returns:
        - hash_block: Devuelve el hash del bloque """

    encoded_block = json.dumps(block, sort_keys = True).encode()
    hash_block = hashlib.sha256(encoded_block).hexdigest()
    return hash_block
  
  def is_chain_valid(self, chain):
    """ Determina si la Blockchain es válida. 
    
    Arguments:
        - chain: Cadena de bloques que contiene toda la 
                  información de las transacciones.
    
    Returns:
        - True/False: Devuelve un booleano en función de la validez de la 
                      Blockchain. (True = Válida, False = Inválida) """
                      
    previous_block = chain[0]
    block_index = 1
    while block_index < len(chain):
        block = chain[block_index]
        if block['previous_hash'] != self.hash(previous_block):
            return False
        previous_proof = previous_block['proof']
        proof = block['proof']
        hash_operation = hashlib.sha256(str(proof**2 - previous_proof**2).encode()).hexdigest()
        if hash_operation[:4] != '0000':
            return False
        previous_block = block
        block_index += 1
    return True
  
  def add_transaction(self, sender, receiver, amount):
    """ Realización de una transacción.
    
    Arguments:
        - sender: Persona que hace la transacción
        - receiver: Persona que recibe la transacción
        - amount: Cantidad de criptomonedas enviadas

    Returns: 
        - Devolución del índice superior al último bloque
    """

    self.transactions.append({'sender'  : sender,
                              'receiver': receiver, 
                              'amount'  : amount})
    previous_block = self.get_previous_block()
    return previous_block['index'] + 1

  def add_node(self, address):
    """ Nuevo nodo en la Blockchain.
    
      Arguments:
        - address: Dirección del nuevo nodo
    """

    parsed_url = urlparse(address)
    self.nodes.add(parsed_url.netloc)
  
  def replace_chain(self):
    """ Remplazo de la cadena por la cadena más larga, 
    siempre y cuando sea válida. """
    
    network = self.nodes
    longest_chain = None
    max_length = len(self.chain)
    for node in network:
        response = requests.get(f'http://{node}/get_chain')
        if response.status_code == 200:
            length = response.json()['length']
            chain = response.json()['chain']
            if length > max_length and self.is_chain_valid(chain):
                max_length = length
                longest_chain = chain
    if longest_chain: 
        self.chain = longest_chain
        return True
    return False

In [20]:
# Minado de un Bloque de la Cadena

# Crear una aplicación web
app = Flask(__name__)
run_with_ngrok(app)  

# Si se obtiene un error 500, actualizar flask, reiniciar spyder y ejecutar la siguiente línea
app.config['JSONIFY_PRETTYPRINT_REGULAR'] = False

# Crear la dirección del nodo en el puerto 5000
node_address = str(uuid4()).replace('-', '')

# Crear una Blockchain
blockchain = Blockchain()


@app.route('/mine_block', methods=['GET'])
def mine_block():
  """ Minado de un nuevo bloque """ 

  previous_block = blockchain.get_previous_block()
  previous_proof = previous_block['proof']
  proof = blockchain.proof_of_work(previous_proof)
  previous_hash = blockchain.hash(previous_block)
  blockchain.add_transaction(sender = node_address, receiver = "Joan Amengual", amount = 10)
  block = blockchain.create_block(proof, previous_hash)
  response = {'message'       : '¡Enhorabuena, nuevo bloque minado!', 
              'index'         : block['index'],
              'timestamp'     : block['timestamp'],
              'proof'         : block['proof'],
              'previous_hash' : block['previous_hash'],
              'transactions'  : block['transactions']}
  return jsonify(response), 200

@app.route('/get_chain', methods=['GET'])
def get_chain():
  """ Obtención de la cadena de bloques al completo """

  response = {'chain'   : blockchain.chain, 
              'length'  : len(blockchain.chain)}
  return jsonify(response), 200

@app.route('/is_valid', methods = ['GET'])
def is_valid():
  """ Comprobación de si la cadena de bloques es válida """

  is_valid = blockchain.is_chain_valid(blockchain.chain)
  if is_valid:
      response = {'message' : 'Todo correcto. La cadena de bloques es válida.'}
  else:
      response = {'message' : 'UPS. La cadena de bloques NO es válida.'}
  return jsonify(response), 200  

@app.route('/add_transaction', methods = ['POST'])
def add_transaction():
  """ Añadir una nueva transacción a la cadena de bloques """

  json = request.get_json()
  transaction_keys = ['sender', 'receiver', 'amount']
  if not all(key in json for key in transaction_keys):
      return 'Faltan algunos elementos de la transacción', 400
  index = blockchain.add_transaction(json['sender'], json['receiver'], json['amount'])
  response = {'message': f'La transacción será añadida al bloque {index}'}
  return jsonify(response), 201
    
# Descentralización de la Cadena de Bloques

# Conectar nuevos nodos
@app.route('/connect_node', methods = ['POST'])
def connect_node():
  json = request.get_json()
  nodes = json.get('nodes')
  if nodes is None: 
      return 'No hay nodos para añadir', 400
  for node in nodes:
      blockchain.add_node(node)
  response = {'message'     : 'Todos los nodos han sido conectados. La Blockchain de JoanCoins contiene ahora los nodos siguientes: ',
              'total_nodes' : list(blockchain.nodes)}
  return jsonify(response), 201

@app.route('/replace_chain', methods = ['GET'])
def replace_chain():
  """ Reemplazar la cadena por la más larga (si es necesario) """

  is_chain_replaced = blockchain.replace_chain()
  if is_chain_replaced:
      response = {'message' : 'Los nodos tenían diferentes cadenas, se ha remplazado por la Blockchain más larga.',
                  'new_chain': blockchain.chain}
  else:
      response = {'message'       : 'Todo correcto. La Blockchain en todos los nodos ya es la más larga.',
                  'actual_chain'  : blockchain.chain}
  return jsonify(response), 200  

In [ ]:
# Ejecución de la app con Google Colab
app.run()

# Ejecución externa a Google colab
#app.run(host = '0.0.0.0', port = 5000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://b3ef-35-227-158-117.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [30/Nov/2021 11:50:42] "GET /get_chain HTTP/1.1" 200 -
127.0.0.1 - - [30/Nov/2021 11:52:23] "GET /mine_block HTTP/1.1" 200 -
127.0.0.1 - - [30/Nov/2021 11:53:08] "GET /get_chain HTTP/1.1" 200 -
127.0.0.1 - - [30/Nov/2021 11:53:40] "GET /is_valid HTTP/1.1" 200 -
127.0.0.1 - - [30/Nov/2021 11:54:36] "POST /add_transaction HTTP/1.1" 201 -
127.0.0.1 - - [30/Nov/2021 11:54:43] "POST /get_chain HTTP/1.1" 405 -
127.0.0.1 - - [30/Nov/2021 11:54:50] "GET /get_chain HTTP/1.1" 200 -
127.0.0.1 - - [30/Nov/2021 11:55:10] "GET /mine_block HTTP/1.1" 200 -
127.0.0.1 - - [30/Nov/2021 11:55:24] "GET /get_chain HTTP/1.1" 200 -
127.0.0.1 - - [30/Nov/2021 11:56:55] "POST /connect_node HTTP/1.1" 201 -
127.0.0.1 - - [30/Nov/2021 11:58:07] "GET /get_chain HTTP/1.1" 200 -
127.0.0.1 - - [30/Nov/2021 11:58:16] "GET /replace_chain HTTP/1.1" 200 -
127.0.0.1 - - [30/Nov/2021 12:01:38] "GET /get_chain HTTP/1.1" 200 -
127.0.0.1 - - [30/Nov/2021 12:01:53] "GET /get_chain HTTP/1.1" 200 -
127.0.0.1 - - [30